In [52]:
import networkx as nx
import torch
from torch_geometric.utils import from_networkx
import torch_geometric.transforms as T
import random
import numpy as np
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv
from torch import Tensor
import torch.nn.functional as F
from torch_geometric.loader import LinkNeighborLoader
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GAE

In [53]:
G = nx.read_gml("/home/schoenstein/these/gnn_postbiblio/graph/graph_light.gml")
data = from_networkx(G)
print(data)

Data(edge_index=[2, 830562], num_nodes=116979)


In [54]:
train = "double split"

if train == "random":
    transform = T.RandomLinkSplit(
        num_val = 0.1,  
        num_test = 0.1,  
        disjoint_train_ratio = 0,  
        neg_sampling_ratio = 1,
        is_undirected = True
    )
    train_data, val_data, test_data = transform(data)
    print(train_data)
    print(val_data)
    print(test_data)

elif train == "double split":
    cc = list(nx.connected_components(G))
    neg_inside = 0
    train_list = []
    val_list = []
    test_list = []
    for c in cc:
        G2 = G.subgraph(c).copy()
        data2 = from_networkx(G2)
        transform = T.RandomLinkSplit(
                num_val = 0.1,  
                num_test = 0.1,  
                disjoint_train_ratio = 0,  
                neg_sampling_ratio = 1.5,
                is_undirected = True
            )
        train_data2, val_data2, test_data2 = transform(data2)
        neg_inside = neg_inside + len(train_data2.edge_label)
        train_list.append(train_data2)
        val_list.append(val_data2)
        test_list.append(test_data2)
    ratio_neg_inside = neg_inside/(len(list(G.edges()))*2)
    print(ratio_neg_inside)
    transform = T.RandomLinkSplit(
            num_val = 0.1,  
            num_test = 0.1,  
            disjoint_train_ratio = 0,  
            neg_sampling_ratio = 1 - ratio_neg_inside,
            is_undirected = True
        )
    train_data3, val_data3, test_data3 = transform(data)

    #pos_train = torch.cat([d.edge_label_index[:, d.edge_label==1] 
                             #for d in train_list], dim=1)
    pos_train = train_data3.edge_label_index[:, train_data3.edge_label == 1]
    #pos_val = torch.cat([d.edge_label_index[:, d.edge_label==1] 
                           #for d in val_list], dim=1)
    pos_val = val_data3.edge_label_index[:, val_data3.edge_label == 1]
    #pos_test = torch.cat([d.edge_label_index[:, d.edge_label==1] 
                            #for d in test_list], dim=1)
    pos_test = test_data3.edge_label_index[:, test_data3.edge_label == 1]
    neg_train1 = torch.cat([d.edge_label_index[:, d.edge_label==0] 
                             for d in train_list], dim=1)
    neg_val1 = torch.cat([d.edge_label_index[:, d.edge_label==0] 
                           for d in val_list], dim=1)
    neg_test1 = torch.cat([d.edge_label_index[:, d.edge_label==0] 
                            for d in test_list], dim=1)
    neg_train2 = train_data3.edge_label_index[:, train_data3.edge_label==0]             
    neg_val2 = val_data3.edge_label_index[:, val_data3.edge_label==0] 
    neg_test2 = test_data3.edge_label_index[:, test_data3.edge_label==0]
    neg_train = torch.cat([neg_train1, neg_train2], dim=1)
    neg_val = torch.cat([neg_val1, neg_val2], dim=1)
    neg_test = torch.cat([neg_test1, neg_test2], dim=1)
    train_data = Data(
        edge_index=train_data3.edge_index,
        num_nodes=data.num_nodes,
        edge_label_index=torch.cat([pos_train, neg_train], dim=1),
        edge_label=torch.cat([
            torch.ones(pos_train.size(1), dtype=torch.long),
            torch.zeros(neg_train.size(1), dtype=torch.long)
        ])
    )
    val_data = Data(
        edge_index=val_data3.edge_index,
        num_nodes=data.num_nodes,
        edge_label_index=torch.cat([pos_val, neg_val], dim=1),
        edge_label=torch.cat([
            torch.ones(pos_val.size(1), dtype=torch.long),
            torch.zeros(neg_val.size(1), dtype=torch.long)
        ])
    )
    test_data = Data(
        edge_index=test_data3.edge_index,
        num_nodes=data.num_nodes,
        edge_label_index=torch.cat([pos_test, neg_test], dim=1),
        edge_label=torch.cat([
            torch.ones(pos_test.size(1), dtype=torch.long),
            torch.zeros(neg_test.size(1), dtype=torch.long)
        ])
    )
    print(train_data)
    print(val_data)
    print(test_data)

/home/schoenstein/.conda/envs/graphe/lib/python3.9/site-packages/torch_geometric/transforms/random_link_split.py:245: UserWarning: There are not enough negative edges to satisfy the provided sampling ratio. The ratio will be adjusted to 0.00.
  warnings.warn(
/home/schoenstein/.conda/envs/graphe/lib/python3.9/site-packages/torch_geometric/transforms/random_link_split.py:245: UserWarning: There are not enough negative edges to satisfy the provided sampling ratio. The ratio will be adjusted to 0.62.
  warnings.warn(
/home/schoenstein/.conda/envs/graphe/lib/python3.9/site-packages/torch_geometric/transforms/random_link_split.py:245: UserWarning: There are not enough negative edges to satisfy the provided sampling ratio. The ratio will be adjusted to 0.33.
  warnings.warn(
/home/schoenstein/.conda/envs/graphe/lib/python3.9/site-packages/torch_geometric/transforms/random_link_split.py:245: UserWarning: There are not enough negative edges to satisfy the provided sampling ratio. The ratio wil

0.8155513977282852
Data(edge_index=[2, 664450], num_nodes=116979, edge_label_index=[2, 728450], edge_label=[728450])
Data(edge_index=[2, 664450], num_nodes=116979, edge_label_index=[2, 85195], edge_label=[85195])
Data(edge_index=[2, 747506], num_nodes=116979, edge_label_index=[2, 90282], edge_label=[90282])


In [55]:
train_data.x = torch.ones((train_data.num_nodes, 1))
val_data.x = train_data.x.clone()
test_data.x = train_data.x.clone()

In [56]:
"""G_train = nx.Graph()
G_train.add_nodes_from(range(train_data.num_nodes))
G_train.add_edges_from(train_data.edge_index.t().tolist())
degree = dict(G_train.degree())
degree_norm = {n: d for n,d in degree.items()}
clustering = nx.clustering(G_train)
degree_tensor = torch.tensor(list(degree_norm.values()), dtype=torch.float32)
clustering_tensor = torch.tensor(list(clustering.values()), dtype=torch.float32)
train_data.x = torch.stack([degree_tensor, clustering_tensor], dim=-1)
val_data.x = train_data.x.clone()
test_data.x = train_data.x.clone()

print(train_data)
print(val_data)
print(test_data)"""

'G_train = nx.Graph()\nG_train.add_nodes_from(range(train_data.num_nodes))\nG_train.add_edges_from(train_data.edge_index.t().tolist())\ndegree = dict(G_train.degree())\ndegree_norm = {n: d for n,d in degree.items()}\nclustering = nx.clustering(G_train)\ndegree_tensor = torch.tensor(list(degree_norm.values()), dtype=torch.float32)\nclustering_tensor = torch.tensor(list(clustering.values()), dtype=torch.float32)\ntrain_data.x = torch.stack([degree_tensor, clustering_tensor], dim=-1)\nval_data.x = train_data.x.clone()\ntest_data.x = train_data.x.clone()\n\nprint(train_data)\nprint(val_data)\nprint(test_data)'

In [57]:
class Encoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        return self.conv2(x, edge_index)

model = GAE(encoder=Encoder(in_channels = train_data.x.shape[1], hidden_channels = 64))
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [58]:
def train_epoch():
    model.train()
    optimizer.zero_grad()
    pred = model.encode(train_data.x, train_data.edge_index)
    pos_edge_index = val_data.edge_label_index[:, val_data.edge_label==1]
    neg_edge_index = val_data.edge_label_index[:, val_data.edge_label==0]
    loss = model.recon_loss(pred, pos_edge_index, neg_edge_index)
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate():
    model.eval()
    pred = model.encode(val_data.x, val_data.edge_index)
    pos_edge_index = val_data.edge_label_index[:, val_data.edge_label==1]
    neg_edge_index = val_data.edge_label_index[:, val_data.edge_label==0]
    loss = model.recon_loss(pred, pos_edge_index, neg_edge_index)
    auc, ap = model.test(pred, pos_edge_index, neg_edge_index)
    return loss, auc, ap
    
#ROC-AUC : la probabilité qu’un positif ait un score plus haut qu’un négatif
#AP : aproxime l'air sous la courbe Precisoion/Recall, plus les positifs sont rares plus un bon score AP est difficile à obtenir

In [59]:
best_val_auc = 0
limit = 10
count = 0
for epoch in range(1, 50):
    loss = train_epoch()
    val_loss, val_auc, val_ap = evaluate()
    print(f"Epoch {epoch:03d}, Loss: {loss:.4f}, Val Loss : {val_loss:.4f}, Val AUC: {val_auc:.4f}, Val AP: {val_ap:.4f}")
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        count = 0
    else:
        count =  count + 1
        if count >= limit:
            print("Early stop")
            break

Epoch 001, Loss: 1.6557, Val Loss : 1.4185, Val AUC: 0.4447, Val AP: 0.4950
Epoch 002, Loss: 1.4185, Val Loss : 1.3910, Val AUC: 0.4419, Val AP: 0.4919
Epoch 003, Loss: 1.3910, Val Loss : 1.3895, Val AUC: 0.4523, Val AP: 0.5070
Epoch 004, Loss: 1.3895, Val Loss : 1.3941, Val AUC: 0.4605, Val AP: 0.5188
Epoch 005, Loss: 1.3941, Val Loss : 1.4013, Val AUC: 0.4623, Val AP: 0.5212
Epoch 006, Loss: 1.4013, Val Loss : 1.4051, Val AUC: 0.4626, Val AP: 0.5213
Epoch 007, Loss: 1.4051, Val Loss : 1.4039, Val AUC: 0.4624, Val AP: 0.5205
Epoch 008, Loss: 1.4039, Val Loss : 1.4000, Val AUC: 0.4619, Val AP: 0.5191
Epoch 009, Loss: 1.4000, Val Loss : 1.3962, Val AUC: 0.4598, Val AP: 0.5162
Epoch 010, Loss: 1.3962, Val Loss : 1.3932, Val AUC: 0.4580, Val AP: 0.5132
Epoch 011, Loss: 1.3932, Val Loss : 1.3911, Val AUC: 0.4589, Val AP: 0.5125
Epoch 012, Loss: 1.3911, Val Loss : 1.3897, Val AUC: 0.4663, Val AP: 0.5171
Epoch 013, Loss: 1.3897, Val Loss : 1.3888, Val AUC: 0.4717, Val AP: 0.5214
Epoch 014, L

random no attri

In [60]:
Epoch 001, Loss: 1.6786, Val Loss : 1.4296, Val AUC: 0.5264, Val AP: 0.5267
Epoch 002, Loss: 1.4296, Val Loss : 1.3946, Val AUC: 0.5237, Val AP: 0.5239
Epoch 003, Loss: 1.3946, Val Loss : 1.3911, Val AUC: 0.5254, Val AP: 0.5255
Epoch 004, Loss: 1.3911, Val Loss : 1.3929, Val AUC: 0.5281, Val AP: 0.5283
Epoch 005, Loss: 1.3929, Val Loss : 1.3960, Val AUC: 0.5284, Val AP: 0.5289
Epoch 006, Loss: 1.3960, Val Loss : 1.3979, Val AUC: 0.5267, Val AP: 0.5276
Epoch 007, Loss: 1.3979, Val Loss : 1.3978, Val AUC: 0.5240, Val AP: 0.5257
Epoch 008, Loss: 1.3978, Val Loss : 1.3963, Val AUC: 0.5227, Val AP: 0.5258
Epoch 009, Loss: 1.3963, Val Loss : 1.3946, Val AUC: 0.5221, Val AP: 0.5264
Epoch 010, Loss: 1.3946, Val Loss : 1.3932, Val AUC: 0.5197, Val AP: 0.5244
Epoch 011, Loss: 1.3932, Val Loss : 1.3919, Val AUC: 0.5203, Val AP: 0.5247
Epoch 012, Loss: 1.3919, Val Loss : 1.3911, Val AUC: 0.5203, Val AP: 0.5230
Epoch 013, Loss: 1.3911, Val Loss : 1.3904, Val AUC: 0.5200, Val AP: 0.5214
Epoch 014, Loss: 1.3904, Val Loss : 1.3898, Val AUC: 0.5193, Val AP: 0.5199
Epoch 015, Loss: 1.3898, Val Loss : 1.3894, Val AUC: 0.5180, Val AP: 0.5181

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (2371608510.py, line 1)

random attri

In [ ]:
Epoch 001, Loss: 26.8638, Val Loss : 16.0256, Val AUC: 0.6189, Val AP: 0.5936
Epoch 002, Loss: 16.0256, Val Loss : 6.6289, Val AUC: 0.6454, Val AP: 0.6597
Epoch 003, Loss: 6.6289, Val Loss : 2.7486, Val AUC: 0.6519, Val AP: 0.6843
Epoch 004, Loss: 2.7486, Val Loss : 1.8491, Val AUC: 0.6528, Val AP: 0.6881
Epoch 005, Loss: 1.8491, Val Loss : 2.0201, Val AUC: 0.6478, Val AP: 0.6849
Epoch 006, Loss: 2.0201, Val Loss : 2.2836, Val AUC: 0.6464, Val AP: 0.6842
Epoch 007, Loss: 2.2836, Val Loss : 2.3185, Val AUC: 0.6465, Val AP: 0.6849
Epoch 008, Loss: 2.3185, Val Loss : 2.1545, Val AUC: 0.6480, Val AP: 0.6872
Epoch 009, Loss: 2.1545, Val Loss : 1.9454, Val AUC: 0.6500, Val AP: 0.6895
Epoch 010, Loss: 1.9454, Val Loss : 1.7805, Val AUC: 0.6528, Val AP: 0.6915
Epoch 011, Loss: 1.7805, Val Loss : 1.6665, Val AUC: 0.6564, Val AP: 0.6939
Epoch 012, Loss: 1.6665, Val Loss : 1.5864, Val AUC: 0.6594, Val AP: 0.6958
Epoch 013, Loss: 1.5864, Val Loss : 1.5246, Val AUC: 0.6617, Val AP: 0.6974
Epoch 014, Loss: 1.5246, Val Loss : 1.4767, Val AUC: 0.6654, Val AP: 0.6998
Epoch 015, Loss: 1.4767, Val Loss : 1.4380, Val AUC: 0.6692, Val AP: 0.7020
Epoch 016, Loss: 1.4380, Val Loss : 1.4079, Val AUC: 0.6728, Val AP: 0.7041
Epoch 017, Loss: 1.4079, Val Loss : 1.3841, Val AUC: 0.6758, Val AP: 0.7059
Epoch 018, Loss: 1.3841, Val Loss : 1.3665, Val AUC: 0.6785, Val AP: 0.7075
Epoch 019, Loss: 1.3665, Val Loss : 1.3536, Val AUC: 0.6810, Val AP: 0.7090
Epoch 020, Loss: 1.3536, Val Loss : 1.3439, Val AUC: 0.6836, Val AP: 0.7106
Epoch 021, Loss: 1.3439, Val Loss : 1.3356, Val AUC: 0.6863, Val AP: 0.7125
Epoch 022, Loss: 1.3356, Val Loss : 1.3292, Val AUC: 0.6890, Val AP: 0.7144
Epoch 023, Loss: 1.3292, Val Loss : 1.3237, Val AUC: 0.6917, Val AP: 0.7164
Epoch 024, Loss: 1.3237, Val Loss : 1.3188, Val AUC: 0.6946, Val AP: 0.7184
Epoch 025, Loss: 1.3188, Val Loss : 1.3140, Val AUC: 0.6976, Val AP: 0.7205
Epoch 026, Loss: 1.3140, Val Loss : 1.3102, Val AUC: 0.7007, Val AP: 0.7227
Epoch 027, Loss: 1.3102, Val Loss : 1.3072, Val AUC: 0.7040, Val AP: 0.7250
Epoch 028, Loss: 1.3072, Val Loss : 1.3047, Val AUC: 0.7075, Val AP: 0.7273
Epoch 029, Loss: 1.3047, Val Loss : 1.3024, Val AUC: 0.7111, Val AP: 0.7297
Epoch 030, Loss: 1.3024, Val Loss : 1.2999, Val AUC: 0.7148, Val AP: 0.7322
Epoch 031, Loss: 1.2999, Val Loss : 1.2982, Val AUC: 0.7185, Val AP: 0.7346
Epoch 032, Loss: 1.2982, Val Loss : 1.2966, Val AUC: 0.7223, Val AP: 0.7371
Epoch 033, Loss: 1.2966, Val Loss : 1.2952, Val AUC: 0.7262, Val AP: 0.7396
Epoch 034, Loss: 1.2952, Val Loss : 1.2939, Val AUC: 0.7303, Val AP: 0.7423
Epoch 035, Loss: 1.2939, Val Loss : 1.2927, Val AUC: 0.7345, Val AP: 0.7450
Epoch 036, Loss: 1.2927, Val Loss : 1.2916, Val AUC: 0.7388, Val AP: 0.7478
Epoch 037, Loss: 1.2916, Val Loss : 1.2906, Val AUC: 0.7433, Val AP: 0.7508
Epoch 038, Loss: 1.2906, Val Loss : 1.2896, Val AUC: 0.7478, Val AP: 0.7537
Epoch 039, Loss: 1.2896, Val Loss : 1.2887, Val AUC: 0.7524, Val AP: 0.7567
Epoch 040, Loss: 1.2887, Val Loss : 1.2879, Val AUC: 0.7570, Val AP: 0.7598
Epoch 041, Loss: 1.2879, Val Loss : 1.2870, Val AUC: 0.7615, Val AP: 0.7628
Epoch 042, Loss: 1.2870, Val Loss : 1.2862, Val AUC: 0.7659, Val AP: 0.7656
Epoch 043, Loss: 1.2862, Val Loss : 1.2855, Val AUC: 0.7700, Val AP: 0.7683
Epoch 044, Loss: 1.2855, Val Loss : 1.2847, Val AUC: 0.7737, Val AP: 0.7709
Epoch 045, Loss: 1.2847, Val Loss : 1.2844, Val AUC: 0.7772, Val AP: 0.7732
Epoch 046, Loss: 1.2844, Val Loss : 1.2837, Val AUC: 0.7804, Val AP: 0.7755
Epoch 047, Loss: 1.2837, Val Loss : 1.2829, Val AUC: 0.7834, Val AP: 0.7776
Epoch 048, Loss: 1.2829, Val Loss : 1.2822, Val AUC: 0.7863, Val AP: 0.7796
Epoch 049, Loss: 1.2822, Val Loss : 1.2815, Val AUC: 0.7890, Val AP: 0.7815

80 attri

In [ ]:
Epoch 001, Loss: 28.0026, Val Loss : 9.0517, Val AUC: 0.6968, Val AP: 0.7438
Epoch 002, Loss: 9.0517, Val Loss : 3.2539, Val AUC: 0.7038, Val AP: 0.7665
Epoch 003, Loss: 3.2539, Val Loss : 1.8492, Val AUC: 0.7072, Val AP: 0.7720
Epoch 004, Loss: 1.8492, Val Loss : 2.0545, Val AUC: 0.7063, Val AP: 0.7713
Epoch 005, Loss: 2.0545, Val Loss : 2.2569, Val AUC: 0.7091, Val AP: 0.7729
Epoch 006, Loss: 2.2569, Val Loss : 2.1632, Val AUC: 0.7146, Val AP: 0.7765
Epoch 007, Loss: 2.1632, Val Loss : 1.9546, Val AUC: 0.7213, Val AP: 0.7813
Epoch 008, Loss: 1.9546, Val Loss : 1.7694, Val AUC: 0.7274, Val AP: 0.7854
Epoch 009, Loss: 1.7694, Val Loss : 1.6397, Val AUC: 0.7314, Val AP: 0.7882
Epoch 010, Loss: 1.6397, Val Loss : 1.5516, Val AUC: 0.7332, Val AP: 0.7897
Epoch 011, Loss: 1.5516, Val Loss : 1.4888, Val AUC: 0.7333, Val AP: 0.7902
Epoch 012, Loss: 1.4888, Val Loss : 1.4438, Val AUC: 0.7319, Val AP: 0.7895
Epoch 013, Loss: 1.4438, Val Loss : 1.4106, Val AUC: 0.7300, Val AP: 0.7886
Epoch 014, Loss: 1.4106, Val Loss : 1.3830, Val AUC: 0.7280, Val AP: 0.7876
Epoch 015, Loss: 1.3830, Val Loss : 1.3607, Val AUC: 0.7263, Val AP: 0.7866
Epoch 016, Loss: 1.3607, Val Loss : 1.3410, Val AUC: 0.7255, Val AP: 0.7860
Epoch 017, Loss: 1.3410, Val Loss : 1.3232, Val AUC: 0.7259, Val AP: 0.7863
Epoch 018, Loss: 1.3232, Val Loss : 1.3065, Val AUC: 0.7274, Val AP: 0.7875
Epoch 019, Loss: 1.3065, Val Loss : 1.2919, Val AUC: 0.7302, Val AP: 0.7895
Epoch 020, Loss: 1.2919, Val Loss : 1.2783, Val AUC: 0.7339, Val AP: 0.7920
Epoch 021, Loss: 1.2783, Val Loss : 1.2663, Val AUC: 0.7381, Val AP: 0.7949
Epoch 022, Loss: 1.2663, Val Loss : 1.2561, Val AUC: 0.7430, Val AP: 0.7979
Epoch 023, Loss: 1.2561, Val Loss : 1.2472, Val AUC: 0.7482, Val AP: 0.8009
Epoch 024, Loss: 1.2472, Val Loss : 1.2394, Val AUC: 0.7537, Val AP: 0.8040
Epoch 025, Loss: 1.2394, Val Loss : 1.2327, Val AUC: 0.7591, Val AP: 0.8068
Epoch 026, Loss: 1.2327, Val Loss : 1.2269, Val AUC: 0.7644, Val AP: 0.8095
Epoch 027, Loss: 1.2269, Val Loss : 1.2219, Val AUC: 0.7693, Val AP: 0.8120
Epoch 028, Loss: 1.2219, Val Loss : 1.2176, Val AUC: 0.7738, Val AP: 0.8141
Epoch 029, Loss: 1.2176, Val Loss : 1.2147, Val AUC: 0.7776, Val AP: 0.8159
Epoch 030, Loss: 1.2147, Val Loss : 1.2110, Val AUC: 0.7810, Val AP: 0.8176
Epoch 031, Loss: 1.2110, Val Loss : 1.2080, Val AUC: 0.7841, Val AP: 0.8192
Epoch 032, Loss: 1.2080, Val Loss : 1.2053, Val AUC: 0.7870, Val AP: 0.8207
Epoch 033, Loss: 1.2053, Val Loss : 1.2037, Val AUC: 0.7898, Val AP: 0.8222
Epoch 034, Loss: 1.2037, Val Loss : 1.2014, Val AUC: 0.7928, Val AP: 0.8237
Epoch 035, Loss: 1.2014, Val Loss : 1.1992, Val AUC: 0.7958, Val AP: 0.8253
Epoch 036, Loss: 1.1992, Val Loss : 1.1976, Val AUC: 0.7989, Val AP: 0.8270
Epoch 037, Loss: 1.1976, Val Loss : 1.1957, Val AUC: 0.8020, Val AP: 0.8286
Epoch 038, Loss: 1.1957, Val Loss : 1.1940, Val AUC: 0.8050, Val AP: 0.8300
Epoch 039, Loss: 1.1940, Val Loss : 1.1925, Val AUC: 0.8078, Val AP: 0.8314
Epoch 040, Loss: 1.1925, Val Loss : 1.1911, Val AUC: 0.8105, Val AP: 0.8327
Epoch 041, Loss: 1.1911, Val Loss : 1.1902, Val AUC: 0.8130, Val AP: 0.8338
Epoch 042, Loss: 1.1902, Val Loss : 1.1889, Val AUC: 0.8151, Val AP: 0.8347
Epoch 043, Loss: 1.1889, Val Loss : 1.1877, Val AUC: 0.8168, Val AP: 0.8356
Epoch 044, Loss: 1.1877, Val Loss : 1.1866, Val AUC: 0.8184, Val AP: 0.8363
Epoch 045, Loss: 1.1866, Val Loss : 1.1854, Val AUC: 0.8199, Val AP: 0.8370
Epoch 046, Loss: 1.1854, Val Loss : 1.1843, Val AUC: 0.8213, Val AP: 0.8377
Epoch 047, Loss: 1.1843, Val Loss : 1.1832, Val AUC: 0.8227, Val AP: 0.8384
Epoch 048, Loss: 1.1832, Val Loss : 1.1825, Val AUC: 0.8242, Val AP: 0.8391
Epoch 049, Loss: 1.1825, Val Loss : 1.1815, Val AUC: 0.8256, Val AP: 0.8398

80 no attri

In [ ]:
Epoch 001, Loss: 1.6557, Val Loss : 1.4185, Val AUC: 0.4447, Val AP: 0.4950
Epoch 002, Loss: 1.4185, Val Loss : 1.3910, Val AUC: 0.4419, Val AP: 0.4919
Epoch 003, Loss: 1.3910, Val Loss : 1.3895, Val AUC: 0.4523, Val AP: 0.5070
Epoch 004, Loss: 1.3895, Val Loss : 1.3941, Val AUC: 0.4605, Val AP: 0.5188
Epoch 005, Loss: 1.3941, Val Loss : 1.4013, Val AUC: 0.4623, Val AP: 0.5212
Epoch 006, Loss: 1.4013, Val Loss : 1.4051, Val AUC: 0.4626, Val AP: 0.5213
Epoch 007, Loss: 1.4051, Val Loss : 1.4039, Val AUC: 0.4624, Val AP: 0.5205
Epoch 008, Loss: 1.4039, Val Loss : 1.4000, Val AUC: 0.4619, Val AP: 0.5191
Epoch 009, Loss: 1.4000, Val Loss : 1.3962, Val AUC: 0.4598, Val AP: 0.5162
Epoch 010, Loss: 1.3962, Val Loss : 1.3932, Val AUC: 0.4580, Val AP: 0.5132
Epoch 011, Loss: 1.3932, Val Loss : 1.3911, Val AUC: 0.4589, Val AP: 0.5125
Epoch 012, Loss: 1.3911, Val Loss : 1.3897, Val AUC: 0.4663, Val AP: 0.5171
Epoch 013, Loss: 1.3897, Val Loss : 1.3888, Val AUC: 0.4717, Val AP: 0.5214
Epoch 014, Loss: 1.3888, Val Loss : 1.3883, Val AUC: 0.4711, Val AP: 0.5205
Epoch 015, Loss: 1.3883, Val Loss : 1.3880, Val AUC: 0.4664, Val AP: 0.5163
Epoch 016, Loss: 1.3880, Val Loss : 1.3877, Val AUC: 0.4622, Val AP: 0.5116
Epoch 017, Loss: 1.3877, Val Loss : 1.3876, Val AUC: 0.4576, Val AP: 0.5064
Epoch 018, Loss: 1.3876, Val Loss : 1.3875, Val AUC: 0.4514, Val AP: 0.4998
Epoch 019, Loss: 1.3875, Val Loss : 1.3874, Val AUC: 0.4445, Val AP: 0.4930
Epoch 020, Loss: 1.3874, Val Loss : 1.3873, Val AUC: 0.4376, Val AP: 0.4867
Epoch 021, Loss: 1.3873, Val Loss : 1.3872, Val AUC: 0.4314, Val AP: 0.4812
Epoch 022, Loss: 1.3872, Val Loss : 1.3871, Val AUC: 0.4260, Val AP: 0.4766
Epoch 023, Loss: 1.3871, Val Loss : 1.3871, Val AUC: 0.4213, Val AP: 0.4728
Early stop